<a href="https://colab.research.google.com/github/NuryaFahruRosyidin2406/Dicoding_Learn-ML/blob/main/Tugas%20Akhir_Project%20Deploy%20ML%20klasifikasi%20gambar%20hewan%20menggunakan%20TensorFlow%20Lite/Tugas_Akhir_Project_Deploy_ML_klasifikasi_gambar_hewan_air_menggunakan_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Tugas Akhir Klasifikasi Hewan Air

## Menyiapkan Library

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import pathlib

## Membaca penyimpanan dataset di GitHub

In [19]:
!git clone https://github.com/NuryaFahruRosyidin2406/Dicoding_Learn-ML.git

fatal: destination path 'Dicoding_Learn-ML' already exists and is not an empty directory.


## Preprocessing Data

1.   Melakukan image augmentation.
2.   Mempersiapkan data latih dan data evaluasi.
3.   Memberi label pada dataset secara otomatis.

In [20]:
TRAINING_DIR = "/content/Dicoding_Learn-ML/Tugas Akhir_Project Deploy ML klasifikasi gambar hewan menggunakan TensorFlow Lite/dataset_hewan_air/train"
VALIDATION_DIR = "/content/Dicoding_Learn-ML/Tugas Akhir_Project Deploy ML klasifikasi gambar hewan menggunakan TensorFlow Lite/dataset_hewan_air/test"

train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 30,
    shear_range = 0.2,
    zoom_range = 0.2,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

Found 10969 images belonging to 23 classes.


## Melihat class nya untuk train

In [21]:
# Mendapatkan kelas-kelas dan indeksnya dari generator
class_labels_train = train_generator.class_indices

# Mencetak kelas-kelas dan indeksnya
print(class_labels_train)

{'Clams': 0, 'Corals': 1, 'Crabs': 2, 'Dolphin': 3, 'Eel': 4, 'Fish': 5, 'Jelly Fish': 6, 'Lobster': 7, 'Nudibranchs': 8, 'Octopus': 9, 'Otter': 10, 'Penguin': 11, 'Puffers': 12, 'Sea Rays': 13, 'Sea Urchins': 14, 'Seahorse': 15, 'Seal': 16, 'Sharks': 17, 'Shrimp': 18, 'Squid': 19, 'Starfish': 20, 'Turtle_Tortoise': 21, 'Whale': 22}


## Melihat class nya untuk validasi

In [22]:
validation_datagen = ImageDataGenerator(
    rescale=1.0/255
)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                            batch_size=16,
                                                            class_mode='categorical',
                                                            target_size=(224, 224))

Found 2742 images belonging to 23 classes.


In [23]:
# Mendapatkan kelas-kelas dan indeksnya dari generator
class_labels_validasi = validation_generator.class_indices

# Mencetak kelas-kelas dan indeksnya
print(class_labels_validasi)

{'Clams': 0, 'Corals': 1, 'Crabs': 2, 'Dolphin': 3, 'Eel': 4, 'Fish': 5, 'Jelly Fish': 6, 'Lobster': 7, 'Nudibranchs': 8, 'Octopus': 9, 'Otter': 10, 'Penguin': 11, 'Puffers': 12, 'Sea Rays': 13, 'Sea Urchins': 14, 'Seahorse': 15, 'Seal': 16, 'Sharks': 17, 'Shrimp': 18, 'Squid': 19, 'Starfish': 20, 'Turtle_Tortoise': 21, 'Whale': 22}


## Memilih kelas-kelas dataset yang akan digunakan

In [24]:
classes = ['Crabs', 'Dolphin', 'Jelly Fish', 'Sea Urchins', 'Starfish', 'Whale']

## Melihat class nya untuk train dari kelas-kelas dataset yang akan digunakan

In [25]:
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224),
                                                    classes=classes)

Found 4085 images belonging to 6 classes.


In [26]:
# Mendapatkan kelas-kelas dan indeksnya dari generator
class_labels_train = train_generator.class_indices

# Mencetak kelas-kelas dan indeksnya
print(class_labels_train)

{'Crabs': 0, 'Dolphin': 1, 'Jelly Fish': 2, 'Sea Urchins': 3, 'Starfish': 4, 'Turtle_Tortoise': 5}


## Melihat class nya untuk validasi dari kelas-kelas dataset yang akan digunakan

In [27]:
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                            batch_size=16,
                                                            class_mode='categorical',
                                                            target_size=(224, 224),
                                                            classes=classes)

Found 1022 images belonging to 6 classes.


In [28]:
# Mendapatkan kelas-kelas dan indeksnya dari generator
class_labels_validasi = validation_generator.class_indices

# Mencetak kelas-kelas dan indeksnya
print(class_labels_validasi)

{'Crabs': 0, 'Dolphin': 1, 'Jelly Fish': 2, 'Sea Urchins': 3, 'Starfish': 4, 'Turtle_Tortoise': 5}


## Membuat Model

!-- Selanjutnya, buat dan latihlah model menggunakan dataset yang telah kita siapkan sebelumnya. Teknik yang akan digunakan kali ini adalah transfer learning memanfaatkan model `MobileNetV2`. --

In [29]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 32)      18464     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 54, 54, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 64)       

In [30]:
# pre_trained_model = MobileNetV2(weights="imagenet", include_top=False,
#                                 input_tensor=Input(shape=(150, 150, 3)))

# for layer in pre_trained_model.layers:
#     layer.trainable = False

# last_output = pre_trained_model.output

In [31]:
# x = tf.keras.layers.Flatten(name="flatten")(last_output)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(32, activation="relu")(x)
# x = tf.keras.layers.Dense(64, activation="relu")(x)
# x = tf.keras.layers.Dense(128, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(23, activation='softmax')(x)

# model = tf.keras.models.Model(pre_trained_model.input, x)

# model.summary()

In [32]:
int_lr = 1e-3
num_epochs = 100

optimizer = tf.optimizers.Adam(learning_rate=int_lr)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9999):
      print("\nAkurasi telah mencapai >99%!")
      self.model.stop_training = True

callbacks = myCallback()

## Melatih Model

In [34]:
H = model.fit(train_generator,
              epochs=num_epochs,
              callbacks=[callbacks],
              validation_data=validation_generator)

Epoch 1/100
128/128 [==============================] - 602s 5s/step - loss: 1.2420 - accuracy: 0.5714 - val_loss: 1.2952 - val_accuracy: 0.5832
Epoch 2/100
128/128 [==============================] - 581s 5s/step - loss: 0.9746 - accuracy: 0.6590 - val_loss: 1.1638 - val_accuracy: 0.6204
Epoch 3/100
128/128 [==============================] - 564s 4s/step - loss: 0.8792 - accuracy: 0.6903 - val_loss: 1.1111 - val_accuracy: 0.6292
Epoch 4/100
128/128 [==============================] - 574s 4s/step - loss: 0.8385 - accuracy: 0.7013 - val_loss: 1.0821 - val_accuracy: 0.6438
Epoch 5/100
128/128 [==============================] - 573s 4s/step - loss: 0.7871 - accuracy: 0.7160 - val_loss: 0.9454 - val_accuracy: 0.6869
Epoch 6/100
128/128 [==============================] - 575s 4s/step - loss: 0.7593 - accuracy: 0.7278 - val_loss: 1.0069 - val_accuracy: 0.6595
Epoch 7/100
128/128 [==============================] - 564s 4s/step - loss: 0.7419 - accuracy: 0.7354 - val_loss: 1.0188 - val_accuracy:

KeyboardInterrupt: 